In [ ]:
# essentials
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os, sys
from scipy.stats import gamma, beta

# bayesflow
sys.path.append(os.path.abspath(os.path.join('../../BayesFlow')))
from bayesflow.diagnostics import plot_sbc_ecdf, plot_sbc_histograms

In [ ]:
sys.path.append("../src")
from networks_10092022 import DynamicGaussianNetworkJoint
from priors import diffusion_prior, random_walk_prior
from micro_models import dynamic_batch_diffusion, diffusion_trial, fast_dm_simulate
from macro_models import random_walk_shared_var, random_walk
from context import generate_design_matrix
from transformations import scale_z, unscale_z

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# post_samples      : np.ndarray of shape (n_data_sets, n_post_draws, n_params)
#                     The posterior draws obtained from n_data_sets

# prior_samples     : np.ndarray of shape (n_data_sets, n_params)
#                     The prior draws obtained for generating n_data_sets

In [ ]:
N_OBS         = 3200
N_PARAMS      = 6
N_SBC         = 2000
N_SAMPLES_SBC = 100

TIME_SLICES = np.array([1, 800, 1600, 2400, 3200]) - 1

In [ ]:
MACRO_MEAN  = beta(a=1, b=25).mean()
MACRO_STD   = beta(a=1, b=25).std()
MICRO_MEANS = [1.75, 1.75, 1.75, 1.75, 1.7, 1] # calculated based on 10000 simulated theta_1:3200
MICRO_STDS   = [1.5, 1.5, 1.5, 1.5, 1.25, 1] # calculated based on 10000 simulated theta_1:3200

In [ ]:
def generator_fun(batch_size, T):
    theta = diffusion_prior(batch_size, n_cond=N_PARAMS-2)
    eta = random_walk_prior(batch_size, N_PARAMS)
    theta_t = random_walk(theta, eta, T)
    context = generate_design_matrix(batch_size, T)
    rt = dynamic_batch_diffusion(theta_t, context).astype(np.float32)
    x = tf.concat((rt, to_categorical(context[:, :, np.newaxis])), axis=-1)

    eta_z = scale_z(eta, MACRO_MEAN, MACRO_STD)
    
    theta_t_z = theta_t.copy()
    for i in range(theta_t.shape[0]):
        theta_t_z[i] =  scale_z(theta_t[i], MICRO_MEANS,  MICRO_STDS)

    return eta_z.astype(np.float32), theta_t_z.astype(np.float32), x

In [ ]:
network_settings = {
    'embedding_lstm_units' : 512, 
    'embedding_gru_units': 512,
    'embedding_dense_args': dict(units=256, activation='selu', kernel_initializer='lecun_normal'),
    'posterior_dense_args': dict(units=128, activation='selu', kernel_initializer='lecun_normal'),
    'n_micro_params': 6,
    'n_macro_params': 6
}
network = DynamicGaussianNetworkJoint(network_settings)

In [ ]:
network.load_weights('../trained_networks/full_dynamic_dm_3200_joint')

## SBC

In [ ]:
%%time
# generate data
eta_z, theta_t_z, data = generator_fun(N_SBC, N_OBS)
print(eta_z.shape)
print(theta_t_z.shape)
print(data.shape)

In [ ]:
# %%time
# eta_z_pred, theta_z_pred = tf.concat([network.sample_n(x, N_SAMPLES_SBC)
#                          for x in tf.split(data, 10, axis=0)], axis=0)

In [ ]:
counter = 0
for x in tf.split(data, 250, axis=0):
    if counter == 0:
        eta_z_pred, theta_z_pred = network.sample_n(x, N_SAMPLES_SBC)
    else:
        eta_z_tmp, theta_z_tmp = network.sample_n(x, N_SAMPLES_SBC)
        eta_z_pred = np.concatenate((eta_z_pred, eta_z_tmp), axis=1)
        theta_z_pred = np.concatenate((theta_z_pred, theta_z_tmp), axis=1)
    counter += 1
    

In [ ]:
theta_pred = unscale_z(theta_z_pred, MICRO_MEANS, MICRO_STDS)
theta_sim = unscale_z(theta_t_z, MICRO_MEANS, MICRO_STDS)
theta_pred = np.transpose(theta_pred, (1, 0, 2, 3))
print(theta_pred.shape)
print(theta_sim.shape)

In [ ]:
# post_samples      : np.ndarray of shape (n_data_sets, n_post_draws, n_params)
#                     The posterior draws obtained from n_data_sets

# prior_samples     : np.ndarray of shape (n_data_sets, n_params)
#                     The prior draws obtained for generating n_data_sets

In [ ]:
TIME_SLICES = np.array([1, 800, 1600, 2400, 3200]) - 1

In [ ]:
for i in TIME_SLICES:
    plot_sbc_ecdf(theta_pred[:, :, i, :], theta_sim[:, i, :])

In [ ]:
for i in TIME_SLICES:
    plot_sbc_histograms(theta_pred[:, :, i, :], theta_sim[:, i, :])